# Task 2

In [16]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .master("local[*]") \
        .getOrCreate()

### Fixing parquet files bug

In [17]:
import numpy as np
schema = {'order_id': np.int32, 'order_date': str, 'order_customer_id': np.int32, 'order_status': str}

In [18]:
import pandas as pd

In [19]:
orders_csv = pd.read_csv('data-files/orders/part-m-00000', header = None,\
                         names = ['order_id', 'order_date', 'order_customer_id', 'order_status'], \
                         dtype = schema)

In [20]:
orders_csv["order_date"]= pd.to_datetime(orders_csv["order_date"])

In [21]:
orders_csv.dtypes

order_id                      int32
order_date           datetime64[ns]
order_customer_id             int32
order_status                 object
dtype: object

In [22]:
orders_csv.to_parquet('./data-files/orders_parquet/orders_parquet.parquet')

In [23]:
#!parquet-tools show data-files/orders_parquet/orders_parquet.parquet

## Start Here

In [24]:
path_orders_pq = "data-files/orders_parquet/orders_parquet.parquet"

In [25]:
df_orders =  spark.read.format('parquet')\
            .load(path_orders_pq)

In [26]:
df_orders.schema

StructType([StructField('order_id', IntegerType(), True), StructField('order_date', TimestampType(), True), StructField('order_customer_id', IntegerType(), True), StructField('order_status', StringType(), True)])

In [27]:
result_orders = df_orders.filter(df_orders.order_status == "COMPLETE")

In [28]:
from pyspark.sql.functions import to_date
from pyspark.sql import functions as F

In [29]:
result_orders = result_orders.select(F.col('order_id'), to_date(F.col('order_date')).alias('order_date'), \
                     F.col('order_customer_id'), F.col('order_status'))

In [30]:
result_orders.write.mode('Overwrite') \
.option("compression", "gzip") \
.json('C:/Users/DevAdmin/MohyWorkSpace/Retail_Proj/result/scenario2/solution')

In [31]:
orders_json =   spark.read.format('json')\
                .load('Retail_Proj/result/scenario2/solution/*.gz')

In [32]:
orders_json.show()

+-----------------+----------+--------+------------+
|order_customer_id|order_date|order_id|order_status|
+-----------------+----------+--------+------------+
|            12111|2013-07-25|       3|    COMPLETE|
|            11318|2013-07-25|       5|    COMPLETE|
|             7130|2013-07-25|       6|    COMPLETE|
|             4530|2013-07-25|       7|    COMPLETE|
|             2568|2013-07-25|      15|    COMPLETE|
|             2667|2013-07-25|      17|    COMPLETE|
|              333|2013-07-25|      22|    COMPLETE|
|             7562|2013-07-25|      26|    COMPLETE|
|              656|2013-07-25|      28|    COMPLETE|
|             3960|2013-07-25|      32|    COMPLETE|
|             4840|2013-07-25|      35|    COMPLETE|
|             2636|2013-07-25|      45|    COMPLETE|
|            10519|2013-07-25|      56|    COMPLETE|
|             1148|2013-07-25|      63|    COMPLETE|
|             5903|2013-07-25|      65|    COMPLETE|
|             1406|2013-07-25|      67|    COM